In [17]:
import numpy as np
import cfbd
from cfbd.rest import ApiException
import pandas as pd
import sqlite3
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = api_key
configuration.api_key_prefix['Authorization']='Bearer'
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))

In [12]:
def extract_team_data(year,week):
    """
    Extracts team statistics from a list of game data and converts it into a DataFrame.
    
    This function processes each game in the provided list, extracts information about each team,
    including their IDs, names, home/away status, points scored, and other statistical categories.
    The resulting data is then normalized into a DataFrame for easier analysis.
    
    :param game_data: List of game objects, where each game object contains details about the game
                      and its participating teams.
    :return: DataFrame containing extracted team statistics for each game.
    """
    game_stats=[] # Initialize a list to store statistics for each team
   
    game_data = api_instance.get_team_game_stats(year=year, week=week)
    for game in game_data:
        game_id=game.id # Extract the unique identifier for the game
        teams=game.teams # Get the list of teams involved in the game
        
        # Iterate through each team in the game
        for team in teams:
            team_id = team.school_id # Extract the unique identifier for the team
            team_name = team.school # Extract the name of the team
            home_away = team.home_away # Determine if the team is playing at home or away

            team_stats = {
                'game_id': game_id,
                'team_id': team_id,
                'team_name': team_name,
                'home_away': home_away,
                'points': team.points # Extract the points scored by the team
                }
                # Add stats to the dictionary
            for stat in team.stats:
                category = stat.category
                value = stat.stat
                team_stats[category] = value
                # Append the team's stats to the list of game statistics
            game_stats.append(team_stats)
    # Concatenate the weekly data into the final DataFrame
    return pd.json_normalize(game_stats)

In [48]:
# game_stat=api_instance.get_team_game_stats(year=2023, week=1)

In [31]:
allWks=pd.concat([extract_team_data(year=2023,week=i) for i in range(1,17)])

In [32]:
con=sqlite3.connect('collegeFootball.db')
cursor_obj= con.cursor()
cursor_obj.execute("DROP TABLE IF EXISTS teamStats23")
allWks.to_sql('teamStats23',con)
con.commit()
con.close()
con=sqlite3.connect('collegeFootball.db')
p3=pd.read_sql('SELECT * FROM teamStats23;',con)

In [35]:
p3.loc[[p3.team_name=='Michigan']]

KeyError: "None of [Index([(False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, ...)], dtype='object')] are in the [index]"

In [34]:
p3.loc[p3.game_id==401540244]

,index,game_id,team_id,team_name,home_away,points,rushingTDs,passingTDs,kickReturnYards,kickReturnTDs,...,firstDowns,puntReturnYards,puntReturnTDs,puntReturns,tacklesForLoss,defensiveTDs,tackles,sacks,qbHurries,passesDeflected
0,0,401540244,2320,Lamar,home,17,1,1,82,0,...,16,None,None,None,None,None,None,None,None,None
1,1,401540244,70,Idaho,away,42,2,4,0,0,...,20,26,0,3,None,None,None,None,None,None
